In [1]:
!pip install torch torchvision hf_transfer huggingface_hub datasets pillow

In [2]:
import os

os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

from datasets import load_dataset

dataset_id = "imageomics/invasive_plants_hawaii"

# splits are "dorsal", "ventral", "both"
both_dataset = load_dataset(dataset_id, split="both")
both_df = both_dataset.to_pandas()
del both_dataset

both_df.head()

/Users/tripp/school/ai-for-conservation/invasive-plants-ml/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,session,filename,sampling_type,image,site,day,plant,level,view,healthy,...,mechanical_damage,other_remarks,expert_healthy,expert_rust,expert_leaf_miner,expert_other_insect,expert_mechanical_damage,expert_confidence,expert_other_remarks,expert_notes
0,session_1_1_16_2025,DSC00752.png,opportunistic,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,1,16,exp,nan,D,No,...,No,nan,No,Yes,No,No,No,Clear,nan,spots
1,session_1_1_16_2025,DSC00753.png,opportunistic,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,1,16,exp,nan,V,No,...,No,nan,No,Yes,No,No,No,Clear,nan,spots
2,session_1_1_16_2025,DSC00675.png,opportunistic,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,1,16,exp,nan,D,No,...,No,nan,No,Yes,No,No,No,Clear,nan,nan
3,session_1_1_16_2025,DSC00676.png,opportunistic,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,1,16,exp,nan,V,No,...,No,nan,No,Yes,No,No,No,Clear,nan,nan
4,session_1_1_16_2025,DSC00687.png,opportunistic,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,1,16,exp,nan,D,No,...,No,nan,No,Yes,No,No,No,Maybe,nan,nan


In [1]:
from torchvision.transforms.functional import pad
from torchvision import transforms
import numpy as np
import torch
import io
from PIL import Image
from tqdm import tqdm

class SquarePad:
    def __call__(self, image):
        w = image.shape[1]
        h = image.shape[2]
        max_wh = max(w, h)
        hp = int((max_wh - w) / 2)
        vp = int((max_wh - h) / 2)
        padding = (hp, vp, hp, vp)
        return pad(image, padding, 0, 'edge')

mean = torch.tensor([0.485, 0.456, 0.406])
std = torch.tensor([0.229, 0.224, 0.225])

class GreenCrop:
    def __call__(self, image):
        print(image.shape)
        downscale = 16
        mean_pool = torch.nn.functional.adaptive_avg_pool2d(image, (downscale, downscale))
        target_color = (torch.tensor([0.25, 0.5, 0.05]) - mean) / std
        color_diff = mean_pool - target_color[:, None, None]
        kernel = torch.ones((1, 8, 8)) / 64
        similarity = torch.nn.functional.conv2d(color_diff.transpose(0, 2).unsqueeze(0), kernel).transpose(0, 2)[0].squeeze(0)

        max_x, max_y = torch.where(similarity == similarity.max())
        max_x, max_y = max_x[0], max_y[0]
        max_x *= downscale
        max_y *= downscale
        
        left = max(0, max_x - 128)
        top = max(0, max_y - 128)
        right = min(image.shape[1], max_x + 128)
        bottom = min(image.shape[2], max_y + 128)

        return image[:, left:right, top:bottom]



transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std),
    SquarePad(),
    transforms.Resize((1024, 1024)),
    GreenCrop(),
])

def binary_string_to_image(binary_string) -> torch.Tensor:
    file = io.BytesIO(binary_string)
    return transform(Image.open(file))

if not os.path.exists("image_tensors"):
    os.makedirs("image_tensors")

both_df["image_tensor_path"] = ""
# for i in tqdm(range(len(both_df))):
for i in tqdm(range(10)):
    b = both_df["image"].iloc[i]["bytes"]
    image = binary_string_to_image(b)
    both_df["image_tensor_path"].iloc[i] = f"image_tensors/image_{i}.pt"
    with open(both_df["image_tensor_path"].iloc[i], "wb") as f:
        torch.save(image, f)

        un_normalized = image * std + mean
        pil_image = un_normalized.to_pil_image()

        from IPython import display

        display(pil_image)

# both_df.drop(columns=["image"], inplace=True)
both_df.head(10).to_csv("preprocessed_both_df.csv", index=False)

NameError: name 'os' is not defined